In [13]:
#optional installations: 
# !pip install yfinance --upgrade --no-cache-dir
# !pip3 install pandas_datareader
# !pip3 install matplotlib


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# ___library_import_statements___
import pandas as pd

# for pandas_datareader, otherwise it might have issues, sometimes there is some version mismatch
pd.core.common.is_list_like = pd.api.types.is_list_like

# make pandas to print dataframes nicely
pd.set_option('expand_frame_repr', False)  

import pandas_datareader.data as web
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import ssl
from urllib import request


#newest yahoo API 
import yfinance as yahoo_finance

#optional 
yahoo_finance.pdr_override()

%matplotlib inline

In [14]:
def computeRSI (dataframe, time_window):
    diff = dataframe.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)

    return rsi


def stochastics( dataframe, low, high, close, k, d ):
    """
    Fast stochastic calculation
    %K = (Current Close - Lowest Low)/
    (Highest High - Lowest Low) * 100
    %D = 3-day SMA of %K

    Slow stochastic calculation
    %K = %D of fast stochastic
    %D = 3-day SMA of %K

    When %K crosses above %D, buy signal 
    When the %K crosses below %D, sell signal
    """

    df = dataframe.copy()

    # Set minimum low and maximum high of the k stoch
    low_min  = df[low].rolling( window = k ).min()
    high_max = df[high].rolling( window = k ).max()

    # Fast Stochastic
    df['k_fast'] = 100 * (df[close] - low_min)/(high_max - low_min)
    df['d_fast'] = df['k_fast'].rolling(window = d).mean()

    # Slow Stochastic
    df['k_slow'] = df["d_fast"]
    df['d_slow'] = df['k_slow'].rolling(window = d).mean()
    df['d_div'] = df['d_fast'] - df['d_slow']

    return df

def MACD(dataframe,close,span1,span2,sig_span):
    df = dataframe.copy()
    exp1 = df[close].ewm(span=span1, adjust=False).mean()
    exp2 = df[close].ewm(span=span2, adjust=False).mean()
    macd = exp1-exp2
    exp3 = macd.ewm(span=sig_span, adjust=False).mean()
    df['MACD line'] = macd
    df['Sig Line'] = exp3
    df['MACD_Div'] = macd-exp3
#     plt.plot(df.Date, macd, label='MACD', color = '#EBD2BE')
#     plt.plot(df.Date, exp3, label='Signal Line', color='#E5A4CB')
#     plt.legend(loc='upper left')
#     plt.show()
    return df

def CBI_func(ticker,start_time, end_time,trade_date_delta,slope1_delta,slope2_delta):
    # df = dataframe.copy()
    # day30 = df['Date'].iloc[-30]
    # start_time = datetime.datetime(2017, 10, 1)
    # #end_time = datetime.datetime(2019, 1, 20)
    # end_time = datetime.datetime.now().date().isoformat() 
    # start_time = (datetime.datetime.now().date() - datetime.timedelta(days = 90)).isoformat()
    # end_time = (datetime.datetime.now().date() - datetime.timedelta(days = 30)).isoformat()
    # print("Start Date: ", start_time)
    # print("End Date: ", end_time )
    # yahoo gives only daily historical data
    connected = False
    while not connected:
        try:
            # ticker_df = pdr.get_data_yahoo(ticker, start=start_time, end=end_time)
            ticker_df = web.get_data_yahoo(ticker, start=start_time, end=end_time)
            # ticker_df = data.DataReader(ticker, 'google',start=start_time, end=end_time)
            connected = True
            # print('connected to yahoo')
        except Exception as e:
            print("Ticker: ", ticker,"  >>  Error: " + str(e))
            # time.sleep( 5 )
            continue 

    # use numerical integer index instead of date    
    ticker_df = ticker_df.reset_index()
    # print(ticker_df.head(5))
    df = ticker_df
    df['RSI'] = computeRSI(df['Adj Close'], 14)
    df = stochastics( df, 'Low', 'High', 'Close', 14, 3 )
    df = MACD(df,'Close',12,26,9)
    # print(df)
    day0 = df['Date'].iloc[-1*trade_date_delta]
    day1 = df['Date'].iloc[-1*slope1_delta]
    day2 = df['Date'].iloc[-1*slope2_delta]

    # print("day of buying: ",day0)
    current_price = df['Close'].iloc[-1*trade_date_delta]
    # print('df:',df)
    # max_price_30day = df.loc[1:30, 'Close'].max()
    mask = df['Date'].between(day0,end_time)
    max_price_30day = df.loc[mask,'Close'].max()
#     mask = df['Time'].between('15:00', '15:15') & (df['Date'] == '2018-06-14')
# res = df.loc[mask, 'high'].max()
#     max_price_30day = (df['Close'].iloc[-1:-22].max())
    # print(max_price_30day)
    Price_delta = (max_price_30day - current_price)/current_price

    avg_volume = avg_volume_in_window(df, 30)

    #RSI values: 
    AllRSI = np.array([df[df['Date']==day0]['RSI'].values,df[df['Date']==day1]['RSI'].values,df[df['Date']==day2]['RSI'].values])
    #d_slow values: 
    AllSS = np.array([df[df['Date']==day0]['d_slow'].values,df[df['Date']==day1]['d_slow'].values,df[df['Date']==day2]['d_slow'].values])
    #d_fast values
    AllSF = np.array([df[df['Date']==day0]['d_fast'].values,df[df['Date']==day1]['d_fast'].values,df[df['Date']==day2]['d_fast'].values])
    #d_div values
    AllSD = np.array([df[df['Date']==day0]['d_div'].values,df[df['Date']==day1]['d_div'].values,df[df['Date']==day2]['d_div'].values])
    #MACD_Div values
    AllMD = np.array([df[df['Date']==day0]['MACD_Div'].values,df[df['Date']==day1]['MACD_Div'].values,df[df['Date']==day2]['MACD_Div'].values])

    # row = df[df['Date']==end_time]['d_slow'].values
    # print(df['Date'].iloc[-1])
    # print(AllRSI, AllSS, AllSF, AllMD)

    #Algorithm for entry ot exit
    CBIL = [] #Call Buying Indicator List

    #RSI Score
    if AllRSI[0] > 50 and AllRSI[0] < 75:
      CBIL.append((abs(AllRSI[0] - 50)/25).item())
    else: 
      CBIL.append(np.nan)
    
    ##SS score
    SS7DayDiv = AllSS[0] - AllSS[1] #use only when there is an increasing trend
    if AllSS[0] > 20 and AllSS[0]< 40 and SS7DayDiv > 0:
      CBIL.append((abs(AllSS[0]-30)/10).item())
    else:
      CBIL.append(np.nan)
        

    #MACD_Div
    AllMDDiv = abs(AllMD[1]) - abs(AllMD[0])
    if AllMD[0] < 0 and AllMD[1] < 0 and abs(AllMD[1]) > abs(AllMD[0]):
      CBIL.append((1-(abs(AllMD[1]) - abs(AllMD[0]))/abs(AllMD[1])).item())
    else:
      CBIL.append(np.nan)
    
    CBIL.append(current_price)
    CBIL.append(Price_delta)
    CBIL.append(day0)

    CBI = (sum(CBIL[0:2])/3.0)
    
    if pd.notna(CBI):
        print("$$$$$ CBI = ",CBI,"$$$$$")
        print("_________________________________")
        print("RSI = ", AllRSI[0], AllRSI[1],AllRSI[2])
        print("Stochastic Slow =", AllSS[0], AllSS[1], AllSS[2])
        print("Stochastic Fast =", AllSF[0], AllSF[1], AllSF[2])
        print("MACD Div =", AllMD[0], AllMD[1], AllMD[2])
        print("Average Volume =", avg_volume)
        print("_________________________________")       
            
    return CBIL

def avg_volume_in_window(dataframe, num_days):
  df = dataframe.copy()
  df['Date'] = pd.to_datetime(df['Date'])

  start_time = (datetime.datetime.now().date() - datetime.timedelta(days = num_days)).isoformat()
  df = df[(df["Date"] > start_time)]
  return df["Volume"].mean(axis=0)




In [15]:
start_time = (datetime.datetime.now().date() - datetime.timedelta(days = 90)).isoformat()
# end_time = (datetime.datetime.now().date() - datetime.timedelta(days = 30)).isoformat()
end_time = datetime.datetime.now().date().isoformat()
print("Start Date: ", start_time)
print("End Date: ", end_time )

ticker_list = ['DRE']
#CBI_all = []
for ticker in ticker_list: 
    CBIL = CBI_func(ticker, start_time, end_time, 30, 3, 7)

print(CBIL, ";", sum(CBIL[0:2])/3.0)

Start Date:  2020-12-05
End Date:  2021-03-05
[*********************100%***********************]  1 of 1 completed
[0.44864047149342523, nan, nan, 40.0099983215332, 0.0514871647015874, Timestamp('2021-01-21 00:00:00')] ; nan


In [12]:
url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
##START - only needed when  getting SSLVerificationError in local##
context = ssl._create_unverified_context()
response = request.urlopen(url, context=context)
html = response.read()
##END - only needed when  getting SSLVerificationError in local##
table=pd.read_html(html)
sp500 = table[0]
print (sp500)
sp500 = sp500[['Symbol','Security','GICS Sector']]
# sp500 = sp500.head(5)

# filename = 'constituents_csv.csv'
# sp500 = pd.read_csv(filename)
sp500['CBI_RSI']= np.nan
sp500['CBI_SS']= np.nan
sp500['CBI_MACD']= np.nan
sp500['CBI'] = np.nan
sp500['t-30price'] = np.nan
sp500['Price_Delta'] = np.nan
sp500['Trade_Date'] = np.nan
sp500 = sp500[sp500['Symbol'] != "."]
# sp500 = sp500.head(100)

# sp500.groupby(['GICS Sector']).count()
# df2['CBI'] = CBI_func(df2['Symbol'],start_time,end_time)

# start_time = (datetime.datetime.now().date() - datetime.timedelta(days = 90)).isoformat()
# end_time = (datetime.datetime.now().date() - datetime.timedelta(days = 30)).isoformat()
print("Start Date: ", start_time)
print("End Date: ", end_time )

#
sector_list = list(sp500['GICS Sector'].unique())
#ticker_list = list(sp500['Symbol'])
passed_tickers=[]
no_data_tickers = []
for sector in sector_list:
    ticker_list = list(sp500[sp500['GICS Sector']==sector]['Symbol'])
    for ticker in ticker_list:
      try:
        CBIL = CBI_func(ticker, start_time, end_time,30,3,7)
        sp500.loc[sp500['Symbol'] == ticker, ['CBI_RSI','CBI_SS','CBI_MACD','CBI','t-30price','Price_Delta','Trade_Date']] = [CBIL[0], CBIL[1], CBIL[2], sum(CBIL[0:2])/3.0,CBIL[3],CBIL[4],CBIL[5]]
        if np.isnan(sum(CBIL[0:2])/3.0) == True:
          print('Analyzed Security: ',ticker, 'Failed')
          
        else: 
          print('Analyzed Security: ',ticker, 'Passed')
          passed_tickers.append(ticker)
      except:
        print('Analyzed Security: ',ticker, 'No Data')
        no_data_tickers.append(ticker)
        continue


#print("CBI = ", CBI_all)
print(sp500)
print('Tickers Qualified by CBI Criteria: ', passed_tickers)
print('Tickers for which data was not obtained: ', no_data_tickers)
sp500.plot(x='CBI',y='Price_Delta',style='o')

NameError: name 'request' is not defined

In [53]:
print(sp500[sp500['Symbol']=="AKAM"])


   Symbol             Security             GICS Sector  CBI_RSI  CBI_SS  CBI_MACD  CBI  t-30price  Price_Delta  Trade_Date
13   AKAM  Akamai Technologies  Information Technology      NaN     NaN       NaN  NaN        NaN          NaN         NaN
